In [175]:
# pip install customtkinter

In [176]:
from tkinter import *
import tkinter as tk
from tkinter.filedialog import askopenfilename
#NOVO
import customtkinter as s
import os
from PIL import Image, ImageTk
import pm4py
import pandas as pd


## Funções

### função que escolhe o caminho 

In [177]:
def escolhe_arquivo(texto):
    caminho_do_arquivo = askopenfilename(filetypes=[ ("Arquivo Excel", "*.xlsx"),("Arquivo CSV", "*.csv"), ("Arquivo XES", '*.xes')])
    if caminho_do_arquivo:
        label_arquivo.configure(text=f"Arquivo selecionado: {caminho_do_arquivo}") # Calma é pra não estar definido msm

    ##---------------------------------------------------------------TENTANDO FAZER OS BOTOES-----------------------------------------------------------------------
    if texto == 'Grafico Frêquencia':
        cria_grafo_dfg(caminho_do_arquivo)
    elif texto == 'Grafico Rede de Petri':
        cria_grafo_petri_net(caminho_do_arquivo)
    elif texto == 'Gráfico de Performance':
        cria_grafo_duracao(caminho_do_arquivo)
    elif texto == 'Análise':
        filtra_grafo_por_variantes(caminho_do_arquivo)

### função cria o gráfico

In [178]:
def cria_grafo_dfg(caminho):
    #Checando se o arquivo é xlsx ou csv
    tipoArq = caminho.split('.')
    try:
        if tipoArq[-1] in 'csv':
            log = pm4py.format_dataframe(pd.read_csv(caminho, sep= ','), case_id='ID', activity_key='Ocorrência', timestamp_key='Início') # depois arrumar um jeito de o usuario selecionar as colunas case_id activity_key e timestamp_key

        elif tipoArq[-1] in 'xlsx':
            log = pm4py.format_dataframe(pd.read_excel(caminho), case_id='ID', activity_key='Ocorrência', timestamp_key='Início') # depois arrumar um jeito de o usuario selecionar as colunas case_id activity_key e timestamp_key

        elif tipoArq[-1] in 'xes':
            log = pm4py.format_dataframe(pm4py.read_xes(caminho), case_id='ID', activity_key='Ocorrência', timestamp_key='Início') # depois arrumar um jeito de o usuario selecionar as colunas case_id 
    except Exception as error:
        lable_auxImagem.configure(text=f"NÃO CONSEGUI TRANSFORMAR EM GRÁFICO. UMA COLUNA NÃO FOI ENCONTRADA NESSE ARQUIVO.\n erro: {error}")
    ##----------------------------------------------------------------------------------------------------------------------------------------
    log['ID'] = log['ID'].astype(str)

    dfg, start_activities, end_activities = pm4py.discover_dfg(log)
    pm4py.vis.save_vis_dfg(dfg, start_activities, end_activities, "grafico.png")
    
    ##----------------------------------------------------------------------------------------------------------------------------------------
    lable_auxImagem.configure(text="Imagem do DFG salva como 'grafico.png'")

    ##---------------------------------------------------------------DEIXEI PRA DEPOIS-------------------------------------------------------------------------

    exibe_grafico("grafico.png")

In [179]:
def cria_grafo_petri_net(caminho):
    #Checando se o arquivo é xlsx ou csv
    tipoArq = caminho.split('.')
    try:
        if tipoArq[-1] in 'csv':
            log = pm4py.format_dataframe(pd.read_csv(caminho, sep=','), case_id='ID', activity_key='Ocorrência', timestamp_key='Início')

        elif tipoArq[-1] in 'xlsx':
            log = pm4py.format_dataframe(pd.read_excel(caminho), case_id='ID', activity_key='Ocorrência', timestamp_key='Início') # depois arrumar um jeito de o usuario selecionar as colunas case_id activity_key e timestamp_key

        elif tipoArq[-1] in 'xes':
            log = pm4py.format_dataframe(pm4py.read_xes(caminho), case_id='ID', activity_key='Ocorrência', timestamp_key='Início') # depois arrumar um jeito de o usuario selecionar as colunas case_id 
    except Exception as error:
        print("NÃO CONSEGUI TRANSFORMAR EM GRÁFICO. UMA COLUNA NÃO FOI ENCONTRADA NESSE ARQUIVO.\n erro: {error}")
    ##----------------------------------------------------------------------------------------------------------------------------------------
 
    pn, ini, fim = pm4py.discover_petri_net_inductive(log)
    pm4py.vis.save_vis_petri_net(pn, ini, fim, "grafico_PetriNet.png")
    ##----------------------------------------------------------------------------------------------------------------------------------------
    lable_auxImagem.configure(text="Imagem do rede de Petri salva como 'grafico_PetriNet.png'")




    exibe_grafico("grafico_PetriNet.png")

#### não consegui colocar em prática

In [180]:
def cria_grafo_duracao(caminho):
    #Checando se o arquivo é xlsx ou csv
    tipoArq = caminho.split('.')
    try:
        if tipoArq[-1] in 'csv':
            log = pm4py.format_dataframe(pd.read_csv(caminho, sep=','), case_id='ID', activity_key='Ocorrência', timestamp_key='Início') # depois arrumar um jeito de o usuario selecionar as colunas case_id activity_key e timestamp_key

        elif tipoArq[-1] in 'xlsx':
            log = pm4py.format_dataframe(pd.read_excel(caminho), case_id='ID', activity_key='Ocorrência', timestamp_key='Início') # depois arrumar um jeito de o usuario selecionar as colunas case_id activity_key e timestamp_key

        elif tipoArq[-1] in 'xes':
            log = pm4py.format_dataframe(pm4py.read_xes(caminho), case_id='ID', activity_key='Ocorrência', timestamp_key='Início') # depois arrumar um jeito de o usuario selecionar as colunas case_id 
    except Exception as error:
        print("NÃO CONSEGUI TRANSFORMAR EM GRÁFICO. UMA COLUNA NÃO FOI ENCONTRADA NESSE ARQUIVO.\n erro: {error}")
    log['ID'] = log['ID'].astype(str)

    performance_dfg, start_activities, end_activities = pm4py.discover_performance_dfg(log, case_id_key='ID', activity_key='Ocorrência', timestamp_key='Início')
    ################ESSE PARAMETRO COLOCA NO GRÁFICO O TEMPO QUE CADA ATIVIDADE DUROU################
    #########serv_time – (optional) provides the activities’ service times, used to decorate the graph
    
    pm4py.vis.save_vis_performance_dfg(performance_dfg, start_activities, end_activities, "grafico_Duracao.png") #bgcolor='str' tem como escolher a cor de fundo
    ##----------------------------------------------------------------------------------------------------------------------------------------
    lable_auxImagem.configure(text="Imagem do DFG salva como 'grafico_Duracao.png'")

    exibe_grafico("grafico_Duracao.png")

### Filtrando os gráficos (Paths)

In [181]:
def filtra_grafo_por_variantes(caminho):
###################################################################################################################################################################################################
    tipoArq = caminho.split('.')
    try:
        if tipoArq[-1] in 'csv':
            log = pm4py.format_dataframe(pd.read_csv(caminho, sep=','), case_id='ID', activity_key='Ocorrência', timestamp_key='Início') # depois arrumar um jeito de o usuario selecionar as colunas case_id activity_key e timestamp_key

        elif tipoArq[-1] in 'xlsx':
            log = pm4py.format_dataframe(pd.read_excel(caminho), case_id='ID', activity_key='Ocorrência', timestamp_key='Início') # depois arrumar um jeito de o usuario selecionar as colunas case_id activity_key e timestamp_key

        elif tipoArq[-1] in 'xes':
            log = pm4py.format_dataframe(pm4py.read_xes(caminho), case_id='ID', activity_key='Ocorrência', timestamp_key='Início') # depois arrumar um jeito de o usuario selecionar as colunas case_id 
    except Exception as error:
        print("NÃO CONSEGUI TRANSFORMAR EM GRÁFICO. UMA COLUNA NÃO FOI ENCONTRADA NESSE ARQUIVO.\n erro: {error}")

###################################################################################################################################################################################################

    log.fillna({'Início': 0},inplace=True)
    log['Início'] = pd.to_datetime(log['Início'])
    log['ID'] = log['ID'].astype(str)

    filtrado = pm4py.filtering.filter_variants_by_coverage_percentage(log, 0.1, case_id_key='ID', activity_key='Ocorrência', timestamp_key='Início')
    filtrado = filtrado.drop('Unnamed: 0', axis=1)

###################################################################################################################################################################################################

    dfg, start_activities, end_activities = pm4py.discover_dfg(filtrado)
    pm4py.vis.save_vis_dfg(dfg, start_activities, end_activities, "grafico_Duracao_filtrado.png")
    
    ##----------------------------------------------------------------------------------------------------------------------------------------
    lable_auxImagem.configure(text="Imagem do DFG salva como 'grafico_Duracao_filtrado.png'")

    exibe_grafico("grafico_Duracao_filtrado.png")

### Função que pega valor do widget Scale

In [182]:
def pega_valor_scale():
    print('atividades:')
    print(escala1.get())
    print('Paths')
    print(paths.get())

In [183]:
# def cria_grafo_analise(caminho):
    
# def filter_dfg_on_activities_percentage(dfg0, start_activities0, end_activities0, activities_count0, percentage):
# def filter_dfg_on_paths_percentage(dfg0, start_activities0, end_activities0, activities_count0, percentage,
#                                    keep_all_activities=False):

### Função que vai plotar os gráficos

In [184]:
lableimg_atual = None

def exibe_grafico(caminho):
    global lableimg_atual
    caminho = './'+caminho
    imagem = Image.open(caminho)

    # Define o tamanho fixo para todos os gráficos
    tamanho_grafico_largura = 1340
    tamanho_grafico_altura = 250

    # Redimensione a imagem do gráfico para o tamanho fixo
    imagem_redimensionada = imagem.resize((tamanho_grafico_largura, tamanho_grafico_altura))

    # Remova o label antigo, se existir
    if lableimg_atual:
        lableimg_atual.destroy()

    # Crie um novo label para o gráfico
    fig_grafico = s.CTkImage(light_image=imagem_redimensionada, dark_image=imagem_redimensionada, size= (tamanho_grafico_largura, tamanho_grafico_altura))
    lableimg_atual = s.CTkLabel(frame_grafico, text=None, image=fig_grafico)
    lableimg_atual.pack()

## Instanciando a janela

In [185]:
raiz = s.CTk(className='Proces Mining GUI')
raiz.geometry('960x540')

#---------------------------------------------TENTATIVA PNG---------------------------------------------------------
# raiz.title("Janela com Ícone Personalizado")

# # Carregar a imagem do ícone usando PIL
# image = Image.open('C:\\Users\\meduarda\\grupo-2-7\\Notebooks\\Tkinter_resumo\\iconProcessGUI.png')
# icon_image = ImageTk.PhotoImage(image)

# # Definir a imagem como ícone da janela
# raiz.iconphoto(False, icon_image)
#---------------------------------------------TENTATIVA .ICO---------------------------------------------------------

# image = Image.open("iconProcessGUI.png")
# icon_image = ImageTk.PhotoImage(image)

# # Definir a imagem como ícone da janela
# raiz.iconphoto(False, icon_image)

## Frames

### Criando o frame de seleção de arquivos

In [186]:
frame1 = s.CTkFrame(raiz)
frame1.pack(side='top',fill=tk.BOTH)

### Criando o frame Lateral

In [187]:
frame_lateral = s.CTkFrame(raiz)
frame_lateral.pack(side = 'right',pady=0,  fill='y')

### Frame do grafico

In [188]:
frame_grafico = s.CTkFrame(raiz, width=1800, height=1000)
frame_grafico.pack(side='bottom', fill='both', expand=True)


## Widgets

### Botão de seleção de arquivos ###

In [189]:
# arq = PhotoImage(file='iconArq.png', master=frame1)
# arq = arq.subsample(2,2)
# fig1 = Label(frame1, image=arq)
# fig1.image = arq

# botao = Button(frame1, image=arq, command=escolhe_arquivo, bd=0, highlightthickness=0, bg= 'green')
# botao.pack(side='left')

label_arquivo = s.CTkLabel(frame1, text="Nenhum arquivo selecionado")
label_arquivo.pack(pady=10)

lable_auxImagem= s.CTkLabel(frame1, text="Nenhma Imagem Gerada")
lable_auxImagem.pack(pady=10)

### Botões de seleção de Gráfico (TESTE) ###

In [190]:
botaoPerformance = s.CTkButton(frame1, text='Análise',  command=lambda: escolhe_arquivo('Análise'))
botaoPerformance.pack(side='left', pady=10)
botaoFrequencia = s.CTkButton(frame1, text='Grafico Frêquencia', command=lambda: escolhe_arquivo('Grafico Frêquencia'))
botaoFrequencia.pack(side='left', pady=10)

botaoPetriNet = s.CTkButton(frame1, text='Grafico Rede de Petri',  command=lambda: escolhe_arquivo('Grafico Rede de Petri'))
botaoPetriNet.pack(side='left', pady=10)

botaoPerformance = s.CTkButton(frame1, text='Grafico Duração',  command=lambda: escolhe_arquivo('Gráfico de Performance'))
botaoPerformance.pack(side='left', pady=10)

In [191]:

# Criando A lable na lateral direita
# label_direita_sup = s.CTkLabel(frame_lateral)#, padx=40, pady=10, borderwidth=2, relief="solid"
# label_direita_sup.pack(fill=tk.Y, expand=True)

################################################## DUDA
# Criando um frame para cada conjunto de label e escala
frame_escala1 = s.CTkFrame(frame_lateral)
frame_escala1.pack(fill="both", expand=True)

# frame_escala2 = s.CTkFrame(frame_lateral, bg_color='green')
# frame_escala2.pack(side = 'top')

# Adicionando rótulo e escala no primeiro frame
label_escala1 = s.CTkLabel(frame_escala1, text="Activities")
label_escala1.place(relx=0.3, rely=0.1, anchor=tk.CENTER)

escala1 = s.CTkSlider(frame_escala1,from_=0, to=1, orientation='vertical')

escala1.set(100)
escala1.place(relx=0.3, rely=0.5, anchor=tk.CENTER)

# Adicionando rótulo e escala no segundo frame
label_escala2 = s.CTkLabel(frame_escala1, text="Paths")
label_escala2.place(relx=0.7, rely=0.1, anchor=tk.CENTER)

paths = s.CTkSlider(frame_escala1, from_=0, to=1, orientation='vertical', )
paths.set(100)
paths.place(relx=0.7, rely=0.5, anchor=tk.CENTER)


# scale = tk.Scrollbar(frame_lateral, orient=tk.VERTICAL)
# scale.pack()



### Botão de aplicar valores

In [192]:
##########################Para a função escolhe arquivo que vai criar um grafico novo##########################
aplicar = s.CTkButton(frame_lateral, text='Aplicar', command=pega_valor_scale)
aplicar.pack()

### Criando widgets da lable lateral

In [193]:
# Bet

# módulos
from tkinter import ttk

# funções
def on_click():
    # Alterna o estado do botão
    if combo3.winfo_ismapped():
        #button.config(state=tk.DISABLED)
        label_secon.pack_forget()
        combo3.pack_forget()
        print("Botão ativado")
    else:
        #button.config(state=tk.NORMAL)
        label_secon.pack(padx=0, pady=0)
        combo3.pack(padx=0, pady=0)
        print("Botão desativado")

def selecionar_opcao(event=None):
    opcao_selecionada = combo.get()
    print("Opção Selecionada:", opcao_selecionada)

# funções do botão de frequency/performance
def open_frequency():
    label_perf.pack_forget()
    combo2.pack_forget()
    label_freq.pack(padx=0, pady=0)
    combo.pack(padx=0, pady=0)

def open_performance():
    label_freq.pack_forget()
    combo.pack_forget()
    label_perf.pack(padx=0, pady=0)
    combo2.pack(padx=0, pady=0)
    
# B.P.
# LABEL DA PARTE LATERAL INFERIOR (DUDA: COMENTEI PQ NN SEI PQ TEM )
# label_direita_inf = s.CTkLabel(frame_lateral, padx=60, pady=10, )#borderwidth=2, relief="solid"
# label_direita_inf.pack(fill=tk.Y, expand=True)#side = 'bottom',

# FREQUENCY
label_freq = s.CTkLabel(frame_lateral, text="Show Frequency:")

opcoes = ["Absolute frequency","Case frequency", "Max. repetitions", "Case coverage"]
combo = s.CTkComboBox(frame_lateral, values=opcoes, state="readonly")
combo.bind("<<ComboboxSelected>>", selecionar_opcao)

button1 = s.CTkButton(frame_lateral, text='Frequency', command=open_frequency)
button1.pack(side = 'top')#

# PERFORMANCE
label_perf = s.CTkLabel(frame_lateral, text="Show Performance:")

opcoes2 = ["Total duration","Median duration", "Mean duration", "Max. duration","Min. duration"]
combo2 = s.CTkComboBox(frame_lateral, values=opcoes2, state="readonly")
combo2.bind("<<ComboboxSelected>>", selecionar_opcao)

button2 = s.CTkButton(frame_lateral, text='Performance', command=open_performance)
button2.pack()#side = 'top'

# SECONDARY METRICS
# frame_secon = tk.Frame(label_direita_inf2)
# frame_secon.pack()

label_secon = s.CTkLabel(frame_lateral, text="Secondary:")

opcoes3 = ["Absolute frequency","Case frequency", "Max. repetitions", "Case coverage","Total duration","Median duration", "Mean duration", "Max. duration","Min. duration"]
combo3 = s.CTkComboBox(frame_lateral, values=opcoes3, state="readonly")

button3 = s.CTkButton(frame_lateral, text='Add secondary metrics', command=on_click)
button3.pack()#side = 'top'
# tem que sobrepor pois os itens do frame de cima esta desaparendo

## LOOPING DA JANELA PRINCIPAL

In [194]:
raiz.mainloop()